#Common functions for all models

### Installing Packages & Importing Libraries

In [ ]:
!pip install pyreadstat

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import transformers

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from transformers import AutoTokenizer,AutoModelForCausalLM
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Functions for WVS Dataset

In [ ]:
#Loading the WVS file and storing it in a dataframe format

def get_wvs_df():
    wvs_df = pd.read_csv('/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/WVS_Moral.csv') #WVS_Moral is a subset of the full data for just the moral questions
    wvs_df_country_names = pd.read_csv('/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Country_Codes_Names-correct.csv')
    wvs_df = wvs_df.set_index('B_COUNTRY').join(wvs_df_country_names.set_index('B_COUNTRY'), how='left')
    return wvs_df

In [ ]:
#Listing all the countries we are gonna use in our study from the WVS dataset

COUNTRIES_WVS_W7_ALL = [
    'Andorra', 'Argentina', 'Armenia', 'Australia', 'Bangladesh', 'Bolivia', 'Brazil', 'Canada',
    'Chile', 'China', 'Colombia', 'Cyprus', 'Ecuador', 'Egypt', 'Ethiopia', 'Germany', 'Greece',
    'Guatemala', 'Indonesia', 'Iran', 'Iraq', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya',
    'Kyrgyzstan', 'Lebanon', 'Libya', 'Malaysia', 'Maldives', 'Mexico', 'Mongolia', 'Morocco',
    'Myanmar', 'Netherlands', 'New Zealand', 'Nicaragua', 'Nigeria', 'Pakistan', 'Peru',
    'Philippines', 'Romania', 'Russia', 'Singapore', 'South Korea', 'Taiwan ROC', 'Tajikistan',
    'Thailand', 'Tunisia', 'Turkey', 'Ukraine', 'United States', 'Venezuela',
    'Vietnam', 'Zimbabwe'
]

# COUNTRIES_WVS_W7_ALL = [
#     'Andorra', 'Argentina', 'Armenia', 'Australia', 'Bangladesh', 'Bolivia', 'Cyprus', 'Ethiopia', 'Germany', 'Greece',
#     'Guatemala', 'Indonesia', 'Iran', 'Iraq', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya',
#     'Kyrgyzstan', 'Lebanon', 'Libya', 'Malaysia', 'Maldives', 'Mongolia', 'Morocco',
#     'Myanmar', 'Netherlands', 'New Zealand', 'Nicaragua', 'Nigeria', 'Pakistan', 'Peru',
#     'Philippines', 'Romania', 'Russia', 'Singapore', 'South Korea', 'Taiwan ROC', 'Tajikistan',
#     'Thailand', 'Tunisia', 'Turkey', 'Ukraine', 'United States', 'Venezuela',
#     'Vietnam', 'Zimbabwe'
# ]

In [ ]:
#Creating the questions for the WVS dataset

W7_QUESTIONS = ['Q'+str(i) for i in range(177, 196)]

W7_QUESTIONS_TEXT = ['claiming government benefits to which you are not entitled',
                     'avoiding a fare on public transport',
                     'stealing property',
                     'cheating on taxes',
                     'someone accepting a bribe in the course of their duties',
                     'homosexuality',
                     'prostitution',
                     'abortion',
                     'divorce',
                     'sex before marriage',
                     'suicide',
                     'euthanasia',
                     'for a man to beat his wife',
                     'parents beating children',
                     'violence against other people',
                     'terrorism as a political, ideological or religious mean',
                     'having casual sex',
                     'political violence',
                     'death penalty']

QUESTION_WAVES_WVS = {7: (W7_QUESTIONS, W7_QUESTIONS_TEXT)}

In [ ]:
#Scaling the survey results between -1 and 1 and calculating the mean rating for a every survey question and culture

MINUS = 5.5
DIVIDE = 4.5

def get_wvs_ratings(wvs_df, culture, q):
    df = wvs_df[['Country_Names', q]]
    df = df.loc[df['Country_Names'] == culture]
    if len(df) == 0:
        return None

    ratings = df.loc[df[q] > 0][q]
    if ratings.empty:
        ratings = 0
    else:
      ratings = ((ratings - MINUS) / DIVIDE).mean()
    return ratings

In [ ]:
#Generating the WVS prompts

def wvs_gpt2_prompts_ratings_multiple_tokens(wvs_df:pd.DataFrame, culture ='', prompt_mode ='in', wave = 7, ):
    prompts = {}
    culture_prompt = ''
    if len(culture) > 0:
        if prompt_mode == 'in':
            print("The token used is IN")
            culture_prompt = f'In {culture} '
        elif prompt_mode == 'people':
            print("The token used is PEOPLE")
            culture_prompt = f'People in {culture} believe '

    questions, questions_text = QUESTION_WAVES_WVS[wave]

    for q,q_text in zip(questions, questions_text):
        rating_answers = get_wvs_ratings(wvs_df, culture, q) #getting the original ratings

        prompts[q_text] = []
        for (prompt_head_moral, prompt_head_nonmoral) in TOKEN_PAIRS:
            prompt_moral = f'{culture_prompt}{q_text} is {prompt_head_moral}.'
            prompt_nonmoral = f'{culture_prompt}{q_text} is {prompt_head_nonmoral}.'

            prompts[q_text].append((prompt_moral, prompt_nonmoral,rating_answers))

    return prompts

### Functions for PEW Dataset

In [ ]:
#Loading the PEW file, pre-processing it and storing it in a dataframe

def get_pew_df():
    pew_data_original = pd.read_spss('/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Pew Research Global Attitudes Project Spring 2013 Dataset for web.sav')

    filtered_columns = pew_data_original.filter(regex='^Q84[A-H]|COUNTRY').copy()

    filtered_columns.rename(columns={'COUNTRY': 'Country_Names'}, inplace=True)

    replace_map = {
        'Morally acceptable': 1,
        'Not a moral issue': 0,
        'Morally unacceptable': -1,
        'Depends on situation (Volunteered)': 0,
        'Refused': 0,
        "Don't know": 0
    }

    filtered_columns.replace(replace_map, inplace=True)

    for col in filtered_columns.columns[1:]:
        filtered_columns[col] = pd.to_numeric(filtered_columns[col], errors='coerce')

    return filtered_columns

In [ ]:
#Listing all the countries we are gonna use in our study from the PEW dataset

COUNTRIES_PEW_ALL = [
    'United States', 'Czech Republic', 'South Korea', 'Canada', 'France', 'Germany',
    'Spain', 'Mexico', 'Chile', 'Australia', 'Russia', 'Britain', 'Turkey', 'Greece',
    'Egypt', 'Poland', 'Senegal', 'Italy', 'Brazil', 'Lebanon', 'Nigeria', 'Japan',
    'Malaysia', 'Kenya', 'Indonesia', 'Uganda', 'Jordan', 'Argentina', 'Philippines',
    'Tunisia', 'China', 'Pakistan', 'Ghana', 'South Africa', 'Palestinian territories',
    'Israel', 'Bolivia', 'Venezuela', 'El Salvador'
]

In [ ]:
#Creating the questions for the PEW dataset

PEW_QUESTIONS = ['Q84' + chr(i) for i in range(ord('A'), ord('H')+1)]

PEW_QUESTIONS_TEXT = ['using contraceptives',
                      'getting a divorce',
                      'having an abortion',
                      'homosexuality',
                      'drinking alcohol',
                      'married people having an affair',
                      'gambling',
                      'sex between unmarried adults']

QUESTION_WAVES_PEW = {13: (PEW_QUESTIONS, PEW_QUESTIONS_TEXT)}

In [ ]:
#Calculating the mean rating for a every question and culture

def get_pew_ratings(pew_df, culture, q):
    df = pew_df[['Country_Names', q]]
    df = df.loc[df['Country_Names'] == culture]
    if df.empty:
        print("No data found for culture:", culture)
        return None

    mean_rating = df[q].mean()

    if pd.isna(mean_rating):
        print("Problem: Mean calculation resulted in NaN for culture:", culture)
        return None

    return mean_rating

In [ ]:
#Generating the PEW prompts

def pew_gpt2_prompts_ratings_multiple_tokens(pew_df:pd.DataFrame, culture ='', prompt_mode ='in', wave = 13, ):
    prompts = {}
    culture_prompt = ''
    if len(culture) > 0:
        if prompt_mode == 'in':
            culture_prompt = f'In {culture} '
            print("Inside IN")
        elif prompt_mode == 'people':
            culture_prompt = f'People in {culture} believe '
            print("Inside PEOPLE")

    questions, questions_text = QUESTION_WAVES_PEW[wave]

    for q,q_text in zip(questions, questions_text):
        rating_answers = get_pew_ratings(pew_df, culture, q) #getting the original ratings

        prompts[q_text] = []
        for (prompt_head_moral, prompt_head_nonmoral) in TOKEN_PAIRS:
            prompt_moral = f'{culture_prompt}{q_text} is {prompt_head_moral}.'
            prompt_nonmoral = f'{culture_prompt}{q_text} is {prompt_head_nonmoral}.'

            prompts[q_text].append((prompt_moral, prompt_nonmoral,rating_answers))

    return prompts

## Other functions

### normalize_log_prob_diffs() and calculate_correlation()

In [ ]:
#Normalizing the log probability differences in a scale between -1 and 1

def normalize_log_prob_diffs(log_prob_diffs):
    min_log_prob = np.min(log_prob_diffs)
    max_log_prob = np.max(log_prob_diffs)
    normalized_log_probs = 2 * (log_prob_diffs - min_log_prob) / (max_log_prob - min_log_prob) - 1
    return normalized_log_probs

#Calculating the Pearson correlation between model and survey scores

def calculate_correlation(survey_scores, log_prob_diffs):
  if len(survey_scores) != len(log_prob_diffs):
      print(f"Error: Mismatched lengths. Survey scores length: {len(survey_scores)}, Log prob diffs length: {len(log_prob_diffs)}")
      return None, None, None

  normalized_log_probs = normalize_log_prob_diffs(log_prob_diffs)
  try:
        correlation, p_value = pearsonr(survey_scores, normalized_log_probs)
  except Exception as e:
        print("Error during correlation calculation:", e)
        return None, None, None
  return correlation, normalized_log_probs, p_value

### get_batch_last_token_log_prob() function

In [ ]:
#Calculating the log probabilities of each of the 2 tokens

def get_batch_last_token_log_prob(lines, model, tokenizer, use_cuda=False, end_with_period=True):
    eos_token = tokenizer.eos_token or tokenizer.sep_token
    if eos_token is None:
        raise ValueError("Neither eos_token nor sep_token is set in the tokenizer.")

    lines = [line + eos_token for line in lines]  #Appending EOS to the end of each line

    tokenizer.pad_token = tokenizer.eos_token

    tok_moral = tokenizer.batch_encode_plus(lines, return_tensors='pt', padding='longest', add_special_tokens=True)
    input_ids = tok_moral['input_ids']
    attention_mask = tok_moral['attention_mask']
    lines_len = torch.sum(attention_mask, dim=1)

    remove_from_end = 2 if end_with_period else 1  #If there is a period remove it as well as the EOS token else remove only the EOS token
    tokens_wanted = lines_len - remove_from_end

    if use_cuda:
        device = next(model.parameters()).device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        torch.cuda.empty_cache()

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids, return_dict=True)
        logits = outputs.logits
        if use_cuda:
            logits = logits.detach().cpu()

    logits_probs = F.log_softmax(logits, dim=-1)

    batch_indices = torch.arange(input_ids.size(0))
    token_indices = tokens_wanted - 1   #take the index of the token we need. index of 1st element is 0 so we have to do -1
    next_token_indices = input_ids[batch_indices, tokens_wanted].cpu()

    log_probs = logits_probs[batch_indices, token_indices, next_token_indices]

    # for line in lines:
    #     print(f"Processed line: {line}")

    return log_probs

### get_log_prob_difference() function

In [ ]:
#Calculating the average log probability differences between moral and non-moral prompts

def get_log_prob_difference(pairs, moral_index, nonmoral_index, model, tokenizer, use_cuda):
    question_average_lm_score = []
    average_moral_score = []
    average_nonmoral_score = []

    all_prompts = []
    for rating in pairs:
        moral_prompt = rating[moral_index]
        nonmoral_prompt = rating[nonmoral_index]
        all_prompts.append(moral_prompt)
        all_prompts.append(nonmoral_prompt)

    logprobs = get_batch_last_token_log_prob(all_prompts, model, tokenizer, use_cuda)

    for i in range(0, len(logprobs), 2):
        moral_logprob = logprobs[i]
        nonmoral_logprob = logprobs[i + 1]
        lm_score = moral_logprob - nonmoral_logprob

        question_average_lm_score.append(lm_score)
        average_moral_score.append(moral_logprob)
        average_nonmoral_score.append(nonmoral_logprob)

    lm_score = np.mean(question_average_lm_score)
    moral_score = np.mean(average_moral_score)
    nonmoral_score = np.mean(average_nonmoral_score)

    return lm_score, moral_score, nonmoral_score

### compare_token_pair() functions for both datasets

In [ ]:
def compare_wvs_token_pairs(model_name, cultures=None, wave=7, excluding_topics=[],
                            excluding_cultures=[], model=None, tokenizer=None, use_cuda=False, prompt_mode='in'):

    if model is None or tokenizer is None:
        tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
        model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
        if use_cuda:
            model = model.cuda()

    wvs_df = get_wvs_df()

    results_all = []

    for culture in tqdm(cultures):
        if culture in excluding_cultures:
            continue
        prompts = wvs_gpt2_prompts_ratings_multiple_tokens(wvs_df, culture, prompt_mode)

        culture_name = culture if culture else 'universal'
        for question, rating_pairs in prompts.items():
            if any(excluded_topic in question for excluded_topic in excluding_topics):
                continue

            lm_score, moral_log_probs, nonmoral_log_probs = get_log_prob_difference(rating_pairs, 0, 1, model, tokenizer, use_cuda)

            if isinstance(moral_log_probs, torch.Tensor):
              moral_log_probs = moral_log_probs.item()
            if isinstance(nonmoral_log_probs, torch.Tensor):
              nonmoral_log_probs = nonmoral_log_probs.item()
            if isinstance(lm_score, torch.Tensor):
              lm_score = lm_score.item()

            wvs_score = rating_pairs[0][2]  #All token pairs have the same wvs_score

            row = {
                'country': culture_name, 'topic': question, 'wvs_score': wvs_score,
                'moral log prob': moral_log_probs, 'non moral log probs': nonmoral_log_probs,
                'log prob difference': lm_score
            }

            results_all.append(row)

    df = pd.DataFrame(results_all)

    survey_scores = df['wvs_score'].values
    log_prob_diffs = df['log prob difference'].values

    correlation, normalized_log_probs, p_value = calculate_correlation(survey_scores, log_prob_diffs)

    print(normalized_log_probs)

    df['normalized log prob difference'] = normalized_log_probs

    metrics = {
        'Pearson correlation coefficient': correlation,
        'P value': p_value
    }

    return df, metrics

In [ ]:
def compare_pew_token_pairs(model_name, cultures=None, wave=7, excluding_topics=[],
                            excluding_cultures=[], model=None, tokenizer=None, use_cuda=False, prompt_mode='in'):

    if model is None or tokenizer is None:
        tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
        model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
        if use_cuda:
            model = model.cuda()

    pew_df = get_pew_df()

    results_all = []

    for culture in tqdm(cultures):
        if culture in excluding_cultures:
            continue
        prompts = pew_gpt2_prompts_ratings_multiple_tokens(pew_df, culture, prompt_mode)

        culture_name = culture if culture else 'universal'
        for question, rating_pairs in prompts.items():
            if any(excluded_topic in question for excluded_topic in excluding_topics):
                continue

            lm_score, moral_log_probs, nonmoral_log_probs = get_log_prob_difference(rating_pairs, 0, 1, model, tokenizer, use_cuda)

            if isinstance(moral_log_probs, torch.Tensor):
              moral_log_probs = moral_log_probs.item()
            if isinstance(nonmoral_log_probs, torch.Tensor):
              nonmoral_log_probs = nonmoral_log_probs.item()
            if isinstance(lm_score, torch.Tensor):
              lm_score = lm_score.item()

            pew_score = rating_pairs[0][2]  #All token pairs have the same pew_score

            row = {
                'country': culture_name, 'topic': question, 'pew_score': pew_score,
                'moral log prob': moral_log_probs, 'non moral log probs': nonmoral_log_probs,
                'log prob difference': lm_score
            }

            results_all.append(row)

    df = pd.DataFrame(results_all)

    survey_scores = df['pew_score'].values
    log_prob_diffs = df['log prob difference'].values

    correlation, normalized_log_probs, p_value = calculate_correlation(survey_scores, log_prob_diffs)

    df['normalized log prob difference'] = normalized_log_probs

    metrics = {
        'Pearson correlation coefficient': correlation,
        'P value': p_value
    }

    return df, metrics

## Functions only for GPT2 models

In [ ]:
#Loading the GPT2 model and the tokenizer

def get_gpt2_model(modelname = 'gpt2', use_cuda = True, device = 'cuda:0'):  #'gpt2','gpt2-medium','gpt2-large','gpt2-xl'
    model = GPT2LMHeadModel.from_pretrained(modelname)
    tokenizer = GPT2Tokenizer.from_pretrained(modelname)

    if use_cuda and torch.cuda.is_available():
        torch.cuda.empty_cache()
        device = torch.device(device)
        model.cuda(device)

    return tokenizer, model

In [ ]:
def compare_wvs_token_pairs_gpt2(model_name, cultures=None, wave=7, excluding_topics=[],
                                 excluding_cultures=[], model=None, tokenizer=None, use_cuda=False, prompt_mode='in'):

    if model is None or tokenizer is None:
        tokenizer, model = get_gpt2_model(model_name, use_cuda)

    wvs_df = get_wvs_df()

    gpt2_all = []

    for culture in tqdm(cultures):
        if culture in excluding_cultures:
            continue
        prompts = wvs_gpt2_prompts_ratings_multiple_tokens(wvs_df, culture, prompt_mode)

        culture_name = culture if culture else 'universal'
        for question, rating_pairs in prompts.items():
            if any(excluded_topic in question for excluded_topic in excluding_topics):
                continue

            lm_score, moral_log_probs, nonmoral_log_probs = get_log_prob_difference(rating_pairs, 0, 1, model, tokenizer, use_cuda)

            wvs_score = rating_pairs[0][2]  #All token pairs have the same wvs_score

            row = {
                'country': culture_name, 'topic': question, 'wvs_score': wvs_score,
                'moral log prob': moral_log_probs, 'non moral log probs': nonmoral_log_probs,
                'log prob difference': lm_score
            }

            gpt2_all.append(row)

    df = pd.DataFrame(gpt2_all)

    survey_scores = df['wvs_score'].values
    log_prob_diffs = df['log prob difference'].values

    correlation, normalized_log_probs, p_value = calculate_correlation(survey_scores, log_prob_diffs)
    if correlation is not None:
      print("Correlation calculated successfully")
    else:
      print("Failed to calculate correlation due to data length mismatch")

    df['normalized log prob difference'] = normalized_log_probs

    metrics = {
        'Pearson correlation coefficient': correlation,
        'P value': p_value
    }

    return df, metrics

In [ ]:
def compare_pew_token_pairs_gpt2(cultures=None, wave=7, model_name='gpt2',
                                 excluding_topics=[], excluding_cultures=[], model=None, tokenizer=None,
                                 use_cuda=False, prompt_mode='in'):
    if model is None or tokenizer is None:
        tokenizer, model = get_gpt2_model(model_name, use_cuda)

    pew_df = get_pew_df()

    gpt2_all = []

    for culture in tqdm(cultures):
        if culture in excluding_cultures:
            continue
        prompts = pew_gpt2_prompts_ratings_multiple_tokens(pew_df, culture, prompt_mode)

        culture_name = culture if culture else 'universal'
        for question, rating_pairs in prompts.items():
            if any(excluded_topic in question for excluded_topic in excluding_topics):
                continue

            lm_score, moral_log_probs, nonmoral_log_probs = get_log_prob_difference(rating_pairs, 0, 1, model, tokenizer, use_cuda)

            pew_score = rating_pairs[0][2]  #All token pairs have the same pew_score

            row = {
                'country': culture_name, 'topic': question, 'pew_score': pew_score,
                'moral log prob': moral_log_probs, 'non moral log probs': nonmoral_log_probs,
                'log prob difference': lm_score
            }

            gpt2_all.append(row)

    df = pd.DataFrame(gpt2_all)

    survey_scores = df['pew_score'].values
    log_prob_diffs = df['log prob difference'].values

    correlation, normalized_log_probs, p_value = calculate_correlation(survey_scores, log_prob_diffs)
    if correlation is not None:
      print("Correlation calculated successfully")
    else:
      print("Failed to calculate correlation due to data length mismatch")

    df['normalized log prob difference'] = normalized_log_probs

    metrics = {
        'Pearson correlation coefficient': correlation,
        'P value': p_value
    }

    return df, metrics

# Function to create combined df with averaged scores over prompt styles

In [ ]:
def generate_combined_df(df_people, df_in, dataset):

  # now create the combined df

  #Step 1: Concatenate the DataFrames column-wise
  combined_df = pd.concat([df_people, df_in], axis=1)

  # Assuming the 'log prob difference' columns have the same name, differentiate them
  combined_df.columns = [f'{col}_people' if i < len(df_people.columns) else f'{col}_in' for i, col in enumerate(combined_df.columns)]

  # Step 2: Calculate the average log probability difference
  combined_df['average log prob difference'] = (combined_df['log prob difference_people'] + combined_df['log prob difference_in']) / 2

  combined_df['average normalized log prob'] = (combined_df['normalized log prob difference_people'] + combined_df['normalized log prob difference_in']) / 2

  if dataset == 'wvs':

    # Step 3: Remove the 'country_in', 'topic_in', and 'wvs_score_in' columns
    columns_to_remove = ['country_in', 'topic_in', 'wvs_score_in']
    combined_df.drop(columns=columns_to_remove, inplace=True)

    # Step 4: Rename the columns
    combined_df.rename(columns={
      'country_people': 'country',
      'topic_people': 'topic',
      'wvs_score_people': 'wvs_score'
    }, inplace=True)

  elif dataset == 'pew':

    # Step 3: Remove the 'country_in', 'topic_in', and 'wvs_score_in' columns
    columns_to_remove = ['country_in', 'topic_in', 'pew_score_in']
    combined_df.drop(columns=columns_to_remove, inplace=True)

    # Step 4: Rename the columns
    combined_df.rename(columns={
      'country_people': 'country',
      'topic_people': 'topic',
      'pew_score_people': 'pew_score'
    }, inplace=True)


  # Print the resulting DataFrame
  display(combined_df)
  return(combined_df)

  # save_dir = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Mijn data/wvs_w7_QWEN_token_pairs_BOTH-MIJNTJE-FINAL.csv'
  # combined_df.to_csv(save_dir, index=False)


In [ ]:
# # TEST: generate combined df function

# df_QWEN_WVS_in = pd.read_csv('/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Mijn data/wvs_w7_QWEN_token_pairs_in-MIJNTJE-FINAL.csv')
# df_QWEN_WVS_people = pd.read_csv('/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Mijn data/wvs_w7_QWEN_token_pairs_people-MIJNTJE-FINAL.csv')

# combined_df_QWEN = generate_combined_df(df_QWEN_WVS_people, df_QWEN_WVS_in)

In [ ]:
def generate_scores_other(model, cultures=COUNTRIES_WVS_W7_ALL):

  # WVS scores

  if model == 'QWEN':
    model_name = "Qwen/Qwen2-0.5B"

  elif model == 'OPT-125':
    model_name = "facebook/opt-125m"

  elif model == 'BLOOM':
    model_name = "bigscience/bloomz-560m"

  # first 'in' prompt type
  df_model_WVS_in, _ = compare_wvs_token_pairs(model_name=model_name, cultures=COUNTRIES_WVS_W7_ALL, use_cuda=False, prompt_mode='in')
  df_model_WVS_people, _ = compare_wvs_token_pairs(model_name=model_name, cultures=COUNTRIES_WVS_W7_ALL, use_cuda=False, prompt_mode='people')

  save_dir_in = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Mijn data/Final test/wvs_w7_{model}_token_pairs_in-MIJNTJE-FINAL.csv'
  df_model_WVS_in.to_csv(save_dir_in, index=False)

  save_dir_people = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Mijn data/Final test/wvs_w7_{model}_token_pairs_people-MIJNTJE-FINAL.csv'
  df_model_WVS_people.to_csv(save_dir_in, index=False)

  combined_df = generate_combined_df(df_model_WVS_people, df_model_WVS_in, 'wvs')
  save_dir = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Mijn data/Final test/wvs_w7_{model}_token_pairs_BOTH-MIJNTJE-FINAL.csv'
  combined_df.to_csv(save_dir, index=False)

  # PEW scores
  df_model_PEW_in, _ = compare_pew_token_pairs(model_name=model_name, cultures=COUNTRIES_PEW_ALL, use_cuda=False, prompt_mode='in')
  df_model_PEW_people, _ = compare_pew_token_pairs(model_name=model_name, cultures=COUNTRIES_PEW_ALL, use_cuda=False, prompt_mode='people')

  save_dir_in = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Mijn data/Final test/PEW_{model}_token_pairs_in-MIJNTJE-FINAL.csv'
  df_model_PEW_in.to_csv(save_dir_in, index=False)

  save_dir_people = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Mijn data/Final test/PEW_{model}_token_pairs_people-MIJNTJE-FINAL.csv'
  df_model_PEW_people.to_csv(save_dir_in, index=False)

  combined_df = generate_combined_df(df_model_PEW_people, df_model_PEW_in, 'pew')
  save_dir = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Mijn data/Final test/PEW_{model}_token_pairs_BOTH-MIJNTJE-FINAL.csv'
  combined_df.to_csv(save_dir, index=False)

  return combined_df

In [ ]:
def generate_scores_gpt(model, cultures=COUNTRIES_WVS_W7_ALL):

  # WVS scores

  if model == 'gpt2':
    model_name = "gpt2"

  elif model == 'gpt2-medium':
    model_name = "gpt2-medium"

  elif model == 'gpt2-large':
    model_name = "gpt2-large"


  # first 'in' prompt type
  df_model_WVS_in, _ = compare_wvs_token_pairs_gpt2(model_name=model_name, cultures=COUNTRIES_WVS_W7_ALL, use_cuda=False, prompt_mode='in')
  df_model_WVS_people, _ = compare_wvs_token_pairs_gpt2(model_name=model_name, cultures=COUNTRIES_WVS_W7_ALL, use_cuda=False, prompt_mode='people')

  save_dir_in = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Mijn data/Final test/wvs_w7_{model}_token_pairs_in-MIJNTJE-FINAL.csv'
  df_model_WVS_in.to_csv(save_dir_in, index=False)

  save_dir_people = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Mijn data/Final test/wvs_w7_{model}_token_pairs_people-MIJNTJE-FINAL.csv'
  df_model_WVS_people.to_csv(save_dir_in, index=False)

  combined_df = generate_combined_df(df_model_WVS_people, df_model_WVS_in, 'wvs')
  save_dir = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Mijn data/Final test/wvs_w7_{model}_token_pairs_BOTH-MIJNTJE-FINAL.csv'
  combined_df.to_csv(save_dir, index=False)

  # PEW scores
  df_model_PEW_in, _ = compare_pew_token_pairs_gpt2(model_name=model_name, cultures=COUNTRIES_PEW_ALL, use_cuda=False, prompt_mode='in')
  df_model_PEW_people, _ = compare_pew_token_pairs_gpt2(model_name=model_name, cultures=COUNTRIES_PEW_ALL, use_cuda=False, prompt_mode='people')

  save_dir_in = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Mijn data/Final test/PEW_{model}_token_pairs_in-MIJNTJE-FINAL.csv'
  df_model_PEW_in.to_csv(save_dir_in, index=False)

  save_dir_people = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Mijn data/Final test/PEW_{model}_token_pairs_people-MIJNTJE-FINAL.csv'
  df_model_PEW_people.to_csv(save_dir_in, index=False)

  combined_df = generate_combined_df(df_model_PEW_people, df_model_PEW_in, 'pew')
  save_dir = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis/Mijn data/Final test/PEW_{model}_token_pairs_BOTH-MIJNTJE-FINAL.csv'
  combined_df.to_csv(save_dir, index=False)

  return combined_df


In [ ]:
TOKEN_PAIRS = [ ('always justifiable', 'never justifiable'),
                ('right','wrong'),
                ('morally good','morally bad'),
                ('ethically right', 'ethically wrong'),
                ('ethical','unethical')]

# TOKEN_PAIRS = [ ('always justifiable', 'never justifiable')]

In [ ]:
GPT2_scores = generate_scores_gpt('gpt2')
GPT2MEDIUM_scores = generate_scores_gpt('gpt2-medium')
GPT2LARGE_scores = generate_scores_gpt('gpt2-large')

QWEN_scores = generate_scores_other('QWEN')
OPT_scores = generate_scores_other('OPT-125')
BLOOM_scores = generate_scores_other('BLOOM')


config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

 97%|█████████▋| 38/39 [01:17<00:02,  2.01s/it]

Inside PEOPLE


100%|██████████| 39/39 [01:19<00:00,  2.04s/it]

Correlation calculated successfully


,country,topic,pew_score,moral log prob_people,non moral log probs_people,log prob difference_people,normalized log prob difference_people,moral log prob_in,non moral log probs_in,log prob difference_in,normalized log prob difference_in,average log prob difference,average normalized log prob
0,United States,using contraceptives,0.459082,-1.646739,-1.386141,-0.260598,0.383477,-1.765900,-1.305743,-0.460156,0.614612,-0.360377,0.499045
1,United States,getting a divorce,0.132735,-1.686044,-1.417812,-0.268232,0.332364,-2.198805,-1.341704,-0.857101,-0.153035,-0.562666,0.089665
2,United States,having an abortion,-0.296407,-1.671703,-1.446117,-0.225585,0.617914,-1.914635,-1.405840,-0.508796,0.520549,-0.367191,0.569231
3,United States,homosexuality,-0.158683,-1.885259,-1.581471,-0.303789,0.094283,-2.687686,-1.775918,-0.911768,-0.258755,-0.607778,-0.082236
4,United States,drinking alcohol,0.177645,-1.693890,-1.431935,-0.261955,0.374390,-2.043410,-1.257033,-0.786377,-0.016262,-0.524166,0.179064
...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,El Salvador,homosexuality,-0.535354,-2.423604,-2.074045,-0.349559,-0.212184,-2.939208,-1.910817,-1.028391,-0.484291,-0.688975,-0.348237
308,El Salvador,drinking alcohol,-0.714646,-2.038171,-1.709301,-0.328870,-0.073655,-2.010871,-1.300106,-0.710766,0.129961,-0.519818,0.028153
309,El Salvador,married people having an affair,-0.857323,-1.882675,-1.664632,-0.218043,0.668419,-2.194096,-1.563966,-0.630130,0.285901,-0.424087,0.477160
310,El Salvador,gambling,-0.748737,-2.313443,-1.852028,-0.461414,-0.961140,-2.192918,-1.463562,-0.729356,0.094010,-0.595385,-0.433565
